In [1]:
from ollama import chat

In [10]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [2]:
skills = [
    "stack(blue_block, green_block)",
    "stack(blue_block, red_block)",
    "stack(green_block, red_block)",
    "stack(green_block, blue_block)",
    "stack(red_block, blue_block)",
    "stack(red_block, green_block)",
    "pick_first(blue_block)",
    "pick_first(green_block)",
    "pick_first(red_block)",
    "done()"
]

In [ ]:
def generate_skill_list(instruction, skills, model):
    S = []  # Skill list
    n = 0   # Step counter
    selected_skill=""
    while selected_skill!="done()":
        C = {}  # Likelihood scores for each skill
        for skill_name in skills:
            known_likelihoods_text = "\n".join(
            f"- {name}: {val}" for name, val in C.items()
            ) or "None so far"
            prompt = f"""
You are a robotic arm responsible for stacking colored blocks in order.  
You can select from these skills to complete the instruction:

1. pick_first(block): Must be the first command. The argument specifies the block at the bottom of the stack.  
2. stack(bottom_block, top_block): Places 'top_block' on top of 'bottom_block'.  
3. done(): Must be the last command after the stack is completed.

Instruction: {instruction}

Previously executed skills: {S or 'None so far'}

In this step, you are evaluating the likelihood of selecting the skill: {skill_name}

So far in this step, the following skills have been evaluated:
{known_likelihoods_text}

Rules:
- Skills must follow the correct execution order.
- Minimize the number of steps required to complete the stack.
- If no skills have been selected yet, you must begin with pick_first.
- If the skill you are currently evaluating is more likely to be the next step than the previously evaluated ones, assign it a higher value than the values you have already given.
- Return only a decimal number between 0 and 1, rounded to two decimal places.
- Do not include any text or explanation — only the number.
"""
            
            response = chat(model=model, messages=[{"role": "user", "content": prompt}])
            print(prompt)
            print(response["message"]["content"])
            likelihood = float(response["message"]["content"].strip())
            C[skill_name] = likelihood
        # Select the skill with the highest likelihood
        selected_skill = max(C, key=C.get)
        print(f"Selected skill: {selected_skill}")
        S.append(selected_skill)
        n += 1

    return S

instruction = "Stack the red block on top of the green block."
skill_list = generate_skill_list(instruction, skills, model="llama3.1")
print("Generated Skill List with Parameters:", skill_list)


You are a robotic arm responsible for stacking colored blocks in order.  
You can select from these skills to complete the instruction:

1. pick_first(block): Must be the first command. The argument specifies the block at the bottom of the stack.  
2. stack(bottom_block, top_block): Places 'top_block' on top of 'bottom_block'.  
3. done(): Must be the last command after the stack is completed.

Instruction: Stack the red block on top of the green block.

Previously executed skills: None so far

In this step, you are evaluating the likelihood of selecting the skill: stack(blue_block, green_block)

So far in this step, the following skills have been evaluated:
None so far

Rules:
- Skills must follow the correct execution order.
- Minimize the number of steps required to complete the stack.
- If no skills have been selected yet, you must begin with pick_first.
- If the skill you are currently evaluating is more likely to be the next step than the previously evaluated ones, assign it a h

# ChatGPT

In [26]:
def generate_skill_list_gpt(instruction, skills):
    S = []  # Skill list
    n = 0   # Step counter
    selected_skill=""
    while selected_skill!="done()":
        C = {}  # Likelihood scores for each skill
        for skill_name in skills:
            known_likelihoods_text = "\n".join(
            f"- {name}: {val}" for name, val in C.items()
            ) or "None so far"
            prompt = f"""
You are a robotic arm responsible for stacking colored blocks in order.  
You can select from these skills to complete the task:

1. **pick_first(block)**: Must be the first command. The argument specifies the block at the bottom of the stack.  
2. **stack(bottom_block, top_block)**: Places 'top_block' on top of 'bottom_block'.  
3. **done()**: Must be the last command after the stack is completed.

---

**Instruction:** {instruction}
The block on the bottom must be placed first, and the block on top should follow after. If the instruction is to stack the red block on top of the green block, the **green block must be placed first**, followed by the **red block on top**.

**Previously executed skills:** {S or 'None so far'}

---

**In this step, you are evaluating the likelihood of selecting the skill:** {skill_name}

**So far in this step, the following skills have been evaluated with their likelihoods:**
{known_likelihoods_text}

---

### Rules:
- The skills must be performed in the correct order: `pick_first` first, `done` last.
- Minimize the number of steps required to complete the stack.
- If no skills have been selected yet, you must start with `pick_first`.
- If the skill you are currently evaluating is more likely to be the next step than any previously evaluated skills in this step, assign it a **higher** likelihood than the ones already evaluated. If it is less likely, assign it a **lower** likelihood than the previously evaluated skills. 
- All likelihood values must be different from each other.
- The likelihood values must be between 0 and 1, with 0 being impossible and 1 being certain.
- The likelihood values must be rounded to two decimal places.
- Do not include any explanation or additional text — **only the number**.
"""
            completion = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}])
            print(completion.choices[0].message.content)
            likelihood = float(completion.choices[0].message.content.strip())
            C[skill_name] = likelihood
        # Select the skill with the highest likelihood
        selected_skill = max(C, key=C.get)
        print(f"Selected skill: {selected_skill}")
        S.append(selected_skill)
        n += 1

    return S

In [ ]:
# Generate the skill list
instruction = "

skill_list = generate_skill_list_gpt(instruction, skills)

0.10
0.2
0.3
0.4
0.5
0.4
0.6
0.7
0.2
0.01
Selected skill: pick_first(green_block)
0.10
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
0.91
Selected skill: done()
